In [1]:
import pandas as pd
import numpy as np
import pickle

In [ ]:
trial2idx = pickle.load(open('./data/trial2idx.pkl', 'rb'))
npi2idx = pickle.load(open('./data/npi2idx.pkl', 'rb'))
trial2npi = pickle.load(open('./data/trial2npi.pkl', 'rb'))

trial_info = pickle.load(open('./data/trial_info.pkl', 'rb'))

trial_list = list(trial2idx.keys())
trial_date = [trial_info[trial]['start_date'] for trial in trial_list]
# Sort the trial list by start date
trial_list = [trial for _, trial in sorted(zip(trial_date, trial_list))]



# Split the trial list into train, val, and test in a 80/10/10 ratio
train_list = trial_list[:int(len(trial_list)*0.8)]
val_list = trial_list[int(len(trial_list)*0.8):int(len(trial_list)*0.9)]
test_list = trial_list[int(len(trial_list)*0.9):]

print(len(train_list), len(val_list), len(test_list))


3403 425 426


In [18]:
trial_list[-1]

'NCT04003480'

In [ ]:
pickle.dump(train_list, open('./data/zero_shot/train_list.pkl', 'wb'))
pickle.dump(val_list, open('./data/zero_shot/val_list.pkl', 'wb'))
pickle.dump(test_list, open('./data/zero_shot/test_list.pkl', 'wb'))

In [5]:
# Gen atomic files
def gen_atomic_file(trial_list, file_name):
    header = 'user_id:token\titem_id:token'
    row_list = []
    for each_trial in trial_list:
        for each_npi in trial2npi[each_trial]:            
            cur_row = '%s\t%s' % (trial2idx[each_trial], npi2idx[each_npi])
            row_list.append(cur_row)
    with open(file_name, 'w') as f:
        f.write('%s\n' % header)
        for each_row in row_list:
            f.write('%s\n' % each_row)

In [ ]:
gen_atomic_file(train_list, './atomic/trial_zero/trial_zero.train.inter')
gen_atomic_file(val_list, './atomic/trial_zero/trial_zero.val.inter')
gen_atomic_file(test_list, './atomic/trial_zero/trial_zero.test.inter')

In [ ]:
trial2pool = pickle.load(open('./data/trial2pool.pkl', 'rb'))
npi_codes = pickle.load(open('./data/npi_codes.pkl', 'rb'))

trial_condition = pickle.load(open('./data/trial_condition.pkl', 'rb'))
trial_summary = pickle.load(open('./data/trial_summary.pkl', 'rb'))
npi2trial = pickle.load(open('./data/npi2trial.pkl', 'rb'))

In [7]:
trial_condition['NCT00004451'].shape

(1718,)

In [8]:
# Generate clinician embd file
# Normalize embd
npi_mat = np.array([npi_codes[each_npi] for each_npi in npi_codes])
npi_mean, npi_std = np.mean(npi_mat, axis=0), np.std(npi_mat, axis=0)
norm_npi_codes = {}
# for each_npi in npi_codes:
#     norm_npi_codes[each_npi] = (npi_codes[each_npi] - npi_mean) / (npi_std + 1e-7)
for each_npi in npi_codes:
    norm_npi_codes[each_npi] = (npi_codes[each_npi] - np.mean(npi_codes[each_npi])) / (np.std(npi_codes[each_npi]) + 1e-7)


In [9]:
# Generate clinician embd file
# Normalize embd
npi_mat = np.array([npi_codes[each_npi] for each_npi in npi_codes])
npi_mean, npi_std = np.mean(npi_mat, axis=0), np.std(npi_mat, axis=0)
norm_npi_codes2 = {}
for each_npi in npi_codes:
    norm_npi_codes2[each_npi] = (npi_codes[each_npi] - npi_mean) / (npi_std + 1e-7)
# for each_npi in npi_codes:
#     norm_npi_codes[each_npi] = (npi_codes[each_npi] - np.mean(npi_codes[each_npi])) / (np.std(npi_codes[each_npi]) + 1e-7)


In [10]:
# Generate external feature file for trials
header = 'user_id:token\tuser_feat:float_seq\tuser_info:float_seq\tuser_summary:float_seq'
row_list = []
for i in range(len(trial_list)):
    cur_embd = trial2pool[trial_list[i]]
    row = str(i) + '\t'
    for j in range(len(cur_embd)):
        row += str(cur_embd[j]) + ' '
    row = row[:-1]
    
    row += '\t'
    cur_info = trial_condition[trial_list[i]]
    for j in range(len(cur_info)):
        row += str(cur_info[j]) + ' '
    row = row[:-1]
    
    row += '\t'
    cur_sum = trial_summary[trial_list[i]]
    for j in range(len(cur_sum)):
        row += str(cur_sum[j]) + ' '
    row = row[:-1]
    
    row_list.append(row)
with open('../atomic/trial_zero/trial_zero.user', 'w') as f:
    f.write('%s\n' % header)
    for each_row in row_list:
        f.write('%s\n' % each_row)

In [ ]:
# Generate clinician embd file
# Normalize embd
header = 'item_id:token\titem_feat:float_seq\titem_his:float_seq\titem_feat_2:float_seq'

row_list = []
for each_npi in norm_npi_codes:
    if each_npi not in npi2idx:
        continue
    cur_embd = norm_npi_codes[each_npi]
    row = str(npi2idx[each_npi]) + '\t'
    for j in range(len(cur_embd)):
        row += str(cur_embd[j]) + ' '
    row = row[:-1]
    
    row += '\t'
    cur_his = npi2trial[each_npi]
    his_list = []
    for each_trial in cur_his:
        if each_trial in train_list:
            his_list.append(trial2pool[each_trial])
    if len(his_list) == 0:
        his_list.append(np.zeros(768))

    cur_his = np.array(his_list).mean(axis=0)
    for j in range(len(cur_his)):
        row += str(cur_his[j]) + ' '
    row = row[:-1]
    
    row += '\t'
    cur_embd = norm_npi_codes2[each_npi]
    for j in range(len(cur_embd)):
        row += str(cur_embd[j]) + ' '
    row = row[:-1]
    row_list.append(row)
    

with open('./atomic/trial_zero/trial_zero.item', 'w') as f:
    f.write('%s\n' % header)
    for each_row in row_list:
        f.write('%s\n' % each_row)

In [12]:
len(npi2idx), len(npi_codes)

(24524, 26242)

In [13]:
len_list = []
for each_trial in trial2npi:
    len_list.append(len(trial2npi[each_trial]))
np.mean(len_list), np.std(len_list), np.median(len_list)

(15.754113775270334, 27.18856131253795, 6.0)